In [1]:
from Model_Training import *
import warnings
from Bus import Bus
from SimulationEngine import SimulationEngine

warnings.filterwarnings('ignore')

### DISCRETE SIMULATION

In [2]:
buses_model = Buses_model()

In [3]:
##Uncomment this if you dont have the time_models.pkl file
buses_model.generate_model()

Iniciando modelado de la distribución de Tiempos de Viaje...
Error: No valid symbolic solution found (likely m=0) for interval 10. Using x_max as inverse.
Error: No valid symbolic solution found (likely m=0) for interval 13. Using x_max as inverse.
Error: No valid symbolic solution found (likely m=0) for interval 14. Using x_max as inverse.
  -> Modelo de Tiempo A2-GLAXO_0700-0930 entrenado.
Error: No valid symbolic solution found (likely m=0) for interval 13. Using x_max as inverse.
Error: No valid symbolic solution found (likely m=0) for interval 14. Using x_max as inverse.
  -> Modelo de Tiempo A2-GLAXO_1330-1800 entrenado.
  -> Modelo de Tiempo A2-GLAXO_0600-0700 entrenado.
Error: No valid symbolic solution found (likely m=0) for interval 9. Using x_max as inverse.
Error: No valid symbolic solution found (likely m=0) for interval 10. Using x_max as inverse.
Error: No valid symbolic solution found (likely m=0) for interval 11. Using x_max as inverse.
Error: No valid symbolic solutio

# Situación actual

### Intervalo '07:00'-'09:00'

In [6]:
# Define Interval
start_time = '07:00'
end_time = '09:30'

# Initialize Buses

#Obligatory used:
eco_07 = Bus(id='07', start_second=0, start_place='DISNEY', cost=42.6, capacity=37, start_time=start_time, end_time=end_time, samples=80)
eco_50 = Bus(id='50', start_second=0, start_place='A2', cost=24.8, capacity=13, start_time=start_time, end_time=end_time, samples=80)

total_buses = [eco_07, eco_50]

N_BUSES = len(total_buses)

#Models
pax_model = Pax_model()
pax_model.generate_model(n_buses=N_BUSES)

# Run Sim
sim = SimulationEngine(total_buses, pax_model, (start_time, end_time), show_process=False)
sim.setup()
sim.run()
sim.get_metrics()

Generating Passengers (Accumulating Inter-arrival Times)...
Scheduling Buses...
Running Event Loop...

      SIMULATION RESULTS SUMMARY      

>> GENERAL PASSENGER METRICS
Total Passengers Moved: 1606
Global Avg Waiting Time: 12.43 min
Global Max Waiting Time: 41.08 min

>> METRICS BY STOP (ORIGIN)
origin  Avg_Wait_Min  Max_Wait_Min
    A2         12.21         39.46
DISNEY         12.61         37.11
 GLAXO         11.81         41.08

>> BUS UTILIZATION & COST
Bus ID Avg Occupancy  Trips Made Total Cost
    07         18.3%         240 $10,224.00
    50         18.3%         240  $5,952.00


### Intervalo '13:30'-'18:00'

In [8]:
# Define Interval
start_time = '13:30'
end_time = '18:00'

# Initialize Buses (Using your params)

#Obligatory used:
eco_07 = Bus(id='07', start_second=0, start_place='DISNEY', cost=42.6, capacity=37, start_time=start_time, end_time=end_time, samples=80)
eco_50 = Bus(id='50', start_second=0, start_place='A2', cost=24.8, capacity=13, start_time=start_time, end_time=end_time, samples=80)

#Add more buses here to simulate them
total_buses = [eco_07, eco_50]

N_BUSES = len(total_buses)

#Models
pax_model = Pax_model()
pax_model.generate_model(n_buses=N_BUSES)

# Run Sim
sim = SimulationEngine(total_buses, pax_model, (start_time, end_time))
sim.setup()
sim.run()
sim.get_metrics()

Generating Passengers (Accumulating Inter-arrival Times)...
Scheduling Buses...
Running Event Loop...

      SIMULATION RESULTS SUMMARY      

>> GENERAL PASSENGER METRICS
Total Passengers Moved: 2120
Global Avg Waiting Time: 11.33 min
Global Max Waiting Time: 39.18 min

>> METRICS BY STOP (ORIGIN)
origin  Avg_Wait_Min  Max_Wait_Min
    A2         11.54         37.53
DISNEY         11.10         39.18
 GLAXO         11.27         38.03

>> BUS UTILIZATION & COST
Bus ID Avg Occupancy  Trips Made Total Cost
    07          6.4%         240 $10,224.00
    50          6.4%         240  $5,952.00


### Intervalo '06:00'-'07:00'

In [9]:
# Define Interval
start_time = '06:00'
end_time = '07:00'

# Initialize Buses (Using your params)

#Obligatory used:
eco_07 = Bus(id='07', start_second=0, start_place='DISNEY', cost=42.6, capacity=37, start_time=start_time, end_time=end_time, samples=80)
eco_50 = Bus(id='50', start_second=0, start_place='A2', cost=24.8, capacity=13, start_time=start_time, end_time=end_time, samples=80)

#Add more buses here to simulate them
total_buses = [eco_07, eco_50]

N_BUSES = len(total_buses)

#Models
pax_model = Pax_model()
pax_model.generate_model(n_buses=N_BUSES)

# Run Sim
sim = SimulationEngine(total_buses, pax_model, (start_time, end_time))
sim.setup()
sim.run()
sim.get_metrics()

Generating Passengers (Accumulating Inter-arrival Times)...
Scheduling Buses...
Running Event Loop...

      SIMULATION RESULTS SUMMARY      

>> GENERAL PASSENGER METRICS
Total Passengers Moved: 1020
Global Avg Waiting Time: 10.76 min
Global Max Waiting Time: 36.62 min

>> METRICS BY STOP (ORIGIN)
origin  Avg_Wait_Min  Max_Wait_Min
    A2         10.88         33.17
DISNEY         10.63         36.62
 GLAXO         10.38         29.77

>> BUS UTILIZATION & COST
Bus ID Avg Occupancy  Trips Made Total Cost
    07          6.7%         240 $10,224.00
    50          6.7%         240  $5,952.00


### Intervalo '09:30'-'13:30'

In [10]:
# Define Interval
start_time = '06:00'
end_time = '07:00'

# Initialize Buses (Using your params)

#Obligatory used:
eco_07 = Bus(id='07', start_second=0, start_place='DISNEY', cost=42.6, capacity=37, start_time=start_time, end_time=end_time, samples=80)
eco_50 = Bus(id='50', start_second=0, start_place='A2', cost=24.8, capacity=13, start_time=start_time, end_time=end_time, samples=80)

#Add more buses here to simulate them
total_buses = [eco_07, eco_50]

N_BUSES = len(total_buses)

#Models
pax_model = Pax_model()
pax_model.generate_model(n_buses=N_BUSES)

# Run Sim
sim = SimulationEngine(total_buses, pax_model, (start_time, end_time))
sim.setup()
sim.run()
sim.get_metrics()

Generating Passengers (Accumulating Inter-arrival Times)...
Scheduling Buses...
Running Event Loop...

      SIMULATION RESULTS SUMMARY      

>> GENERAL PASSENGER METRICS
Total Passengers Moved: 1032
Global Avg Waiting Time: 10.79 min
Global Max Waiting Time: 35.63 min

>> METRICS BY STOP (ORIGIN)
origin  Avg_Wait_Min  Max_Wait_Min
    A2         10.78         35.35
DISNEY         10.81         34.39
 GLAXO         10.74         35.63

>> BUS UTILIZATION & COST
Bus ID Avg Occupancy  Trips Made Total Cost
    07          6.9%         240 $10,224.00
    50          6.9%         240  $5,952.00


### Intervalo '18:30'-'22:15'

In [11]:
# Define Interval
start_time = '06:00'
end_time = '07:00'

# Initialize Buses (Using your params)

#Obligatory used:
eco_07 = Bus(id='07', start_second=0, start_place='DISNEY', cost=42.6, capacity=37, start_time=start_time, end_time=end_time, samples=80)
eco_50 = Bus(id='50', start_second=0, start_place='A2', cost=24.8, capacity=13, start_time=start_time, end_time=end_time, samples=80)

#Add more buses here to simulate them
total_buses = [eco_07, eco_50]

N_BUSES = len(total_buses)

#Models
pax_model = Pax_model()
pax_model.generate_model(n_buses=N_BUSES)

# Run Sim
sim = SimulationEngine(total_buses, pax_model, (start_time, end_time))
sim.setup()
sim.run()
sim.get_metrics()

Generating Passengers (Accumulating Inter-arrival Times)...
Scheduling Buses...
Running Event Loop...

      SIMULATION RESULTS SUMMARY      

>> GENERAL PASSENGER METRICS
Total Passengers Moved: 1071
Global Avg Waiting Time: 10.15 min
Global Max Waiting Time: 31.69 min

>> METRICS BY STOP (ORIGIN)
origin  Avg_Wait_Min  Max_Wait_Min
    A2          9.90         31.69
DISNEY         10.61         28.34
 GLAXO          9.38         25.04

>> BUS UTILIZATION & COST
Bus ID Avg Occupancy  Trips Made Total Cost
    07          7.1%         240 $10,224.00
    50          7.1%         240  $5,952.00
